In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pkg_resources
import ast

IMG_PATH = pkg_resources.resource_filename(
    "da_for_polymers", "data/input_representation/OPV_Min/manual_frag/"
)

smarts: str = "['[Yb]1:[#6]:[#6]:[#6]2-[#6](=[#8])-[#7]-[#6](=[#8])-[#6]:2:[#6]:1', '[#6]-[#6]1(-[#6])-[#6]-[Gd](-[#6])-[#6]2:[#6]:[#6]:[#6]:[#6]:[#6]:2-1', '[Tb]1:[#6]:[#6]:[Dy]:[#6]:[#6]:1', '[Ho]1-[#6](=[#8])-[#6]2:[#6]:[#6]:[Er]:[#6]:[#6]:2-[#6]-1=[#8]', '[Tm]=[#8]']"
frags: list[str] = ast.literal_eval(smarts)
# Reaction SMARTS
curr_mol = frags[0]
print(curr_mol)
idx = 2
for frag in frags[1:]:
    print(frag)
    atomic_num: int = 64
    recombined = False
    stop_condition = 0
    print(idx, len(frags))
    while not recombined:
        if stop_condition == 50:
            assert False
        elif idx == len(frags):
            try:
                # Order matters
                rxn = AllChem.ReactionFromSmarts('[#{}:1].[#{}:2]>>[C:1]-[C:2]'.format(atomic_num + 1, atomic_num))
                products = rxn.RunReactants([Chem.MolFromSmarts(x) for x in [curr_mol, frag]])
                if products == ():
                    rxn = AllChem.ReactionFromSmarts('[#{}:1].[#{}:2]>>[C:1]-[C:2]'.format(atomic_num, atomic_num + 1))
                    products = rxn.RunReactants([Chem.MolFromSmarts(x) for x in [curr_mol, frag]])
                curr_mol = products[0][0] # fails if products is ()
                curr_mol = Chem.MolToSmarts(curr_mol)
                print(curr_mol)
                recombined = True
            except:
                try: 
                    # Order matters
                    atomic_num_2: int = 64
                    rxn = AllChem.ReactionFromSmarts('[#{}:1].[#{}:2]>>[C:1]-[C:2]'.format(atomic_num_2 + len(frags) - 1, atomic_num_2))
                    products = rxn.RunReactants([Chem.MolFromSmarts(x) for x in [curr_mol, frag]])
                    if products == ():
                        rxn = AllChem.ReactionFromSmarts('[#{}:1].[#{}:2]>>[C:1]-[C:2]'.format(atomic_num_2, atomic_num_2 + len(frags) - 1))
                        products = rxn.RunReactants([Chem.MolFromSmarts(x) for x in [curr_mol, frag]])
                    curr_mol = products[0][0] # fails if products is ()
                    curr_mol = Chem.MolToSmarts(curr_mol)
                    print(curr_mol)
                    recombined = True
                except:
                    atomic_num += 1
                    stop_condition += 1
        else:
            try:
                # Order matters
                rxn = AllChem.ReactionFromSmarts('[#{}:1].[#{}:2]>>[#{}:1]-[C:2]'.format(atomic_num + 1, atomic_num, atomic_num + 1))
                products = rxn.RunReactants([Chem.MolFromSmarts(x) for x in [curr_mol, frag]])
                if products == ():
                    rxn = AllChem.ReactionFromSmarts('[#{}:1].[#{}:2]>>[C:1]-[#{}:2]'.format(atomic_num, atomic_num + 1, atomic_num + 1))
                    products = rxn.RunReactants([Chem.MolFromSmarts(x) for x in [curr_mol, frag]])
                curr_mol = products[0][0] # fails if products is ()
                curr_mol = Chem.MolToSmarts(curr_mol)
                print(curr_mol)
                recombined = True
            except:
                try: 
                    # Order matters
                    atomic_num_2: int = 64
                    rxn = AllChem.ReactionFromSmarts('[#{}:1].[#{}:2]>>[C:1]-[#{}:2]'.format(atomic_num_2 + len(frags) - 1, atomic_num_2, atomic_num_2))
                    products = rxn.RunReactants([Chem.MolFromSmarts(x) for x in [curr_mol, frag]])
                    if products == ():
                        rxn = AllChem.ReactionFromSmarts('[#{}:1].[#{}:2]>>[C:1]-[#{}:2]'.format(atomic_num_2, atomic_num_2 + len(frags) - 1, atomic_num_2 + len(frags) - 1))
                        products = rxn.RunReactants([Chem.MolFromSmarts(x) for x in [curr_mol, frag]])
                    curr_mol = products[0][0] # fails if products is ()
                    curr_mol = Chem.MolToSmarts(curr_mol)
                    print(curr_mol)
                    recombined = True
                except:
                    atomic_num += 1
                    stop_condition += 1
    idx += 1

# replace end dummy group with methyl group
curr_mol = Chem.MolFromSmarts(curr_mol)
ed_mol = Chem.EditableMol(curr_mol)
ed_mol.BeginBatchEdit()
for atom in curr_mol.GetAtoms():
    if atom.GetAtomicNum() >= 100:
        ed_mol.ReplaceAtom(atom.GetIdx(), Chem.Atom(6))
        atom_idx = ed_mol.AddAtom(Chem.Atom(6))
        ed_mol.AddBond(atom.GetIdx(), atom_idx)
ed_mol.CommitBatchEdit()
curr_mol = ed_mol.GetMol()
Chem.SanitizeMol(curr_mol)

for atom in curr_mol.GetAtoms():
    atom.SetAtomMapNum(atom.GetIdx())
drawn = Chem.Draw.MolToFile(curr_mol, IMG_PATH + "manual_rearranged.png", size=(700, 700))

# removes atom map numbering
[a.SetAtomMapNum(0) for a in curr_mol.GetAtoms() if a.GetAtomicNum() != 0]

print(Chem.MolToSmiles(curr_mol))


[Yb]1:[#6]:[#6]:[#6]2-[#6](=[#8])-[#7]-[#6](=[#8])-[#6]:2:[#6]:1
[#6]-[#6]1(-[#6])-[#6]-[Gd](-[#6])-[#6]2:[#6]:[#6]:[#6]:[#6]:[#6]:2-1
2 5


AssertionError: 

In [21]:
smi = "CCCCCCCCc1csc(-c2nnc(-c3cc(CCCCCCCC)c(-c4cc5c(-c6ccc(CC(CCCCCC)CCCCCCCC)s6)c6sccc6c(-c6ccc(CCCCCCCCC(C)CCCCCC)s6)c5s4)s3)o2)c1"
smi2 = "CCCCCCCCc1cc(-c2nnc(~C)o2)sc1-c1cc2c(-c3ccc(CC(CCCCCC)CCCCCCCC)s3)c3sc(-c4sc(~C)cc4CCCCCCCC)cc3c(-c3ccc(CCCCCCCCC(C)CCCCCC)s3)c2s1"
print(Chem.MolFromSmiles(smi2))
for bond in Chem.MolFromSmiles(smi).GetBonds():
    print(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())
    print("bondtype", bond.GetBondType())

0 1
bondtype SINGLE
1 2
bondtype SINGLE
2 3
bondtype SINGLE
3 4
bondtype SINGLE
4 5
bondtype SINGLE
5 6
bondtype SINGLE
6 7
bondtype SINGLE
7 8
bondtype SINGLE
8 9
bondtype AROMATIC
9 10
bondtype AROMATIC
10 11
bondtype AROMATIC
11 12
bondtype SINGLE
12 13
bondtype AROMATIC
13 14
bondtype AROMATIC
14 15
bondtype AROMATIC
15 16
bondtype SINGLE
16 17
bondtype AROMATIC
17 18
bondtype AROMATIC
18 19
bondtype SINGLE
19 20
bondtype SINGLE
20 21
bondtype SINGLE
21 22
bondtype SINGLE
22 23
bondtype SINGLE
23 24
bondtype SINGLE
24 25
bondtype SINGLE
25 26
bondtype SINGLE
18 27
bondtype AROMATIC
27 28
bondtype SINGLE
28 29
bondtype AROMATIC
29 30
bondtype AROMATIC
30 31
bondtype AROMATIC
31 32
bondtype SINGLE
32 33
bondtype AROMATIC
33 34
bondtype AROMATIC
34 35
bondtype AROMATIC
35 36
bondtype SINGLE
36 37
bondtype SINGLE
37 38
bondtype SINGLE
38 39
bondtype SINGLE
39 40
bondtype SINGLE
40 41
bondtype SINGLE
41 42
bondtype SINGLE
42 43
bondtype SINGLE
37 44
bondtype SINGLE
44 45
bondtype SINGLE